In [1]:
# Include main project path in the system path for python
import sys
import os
sys.path.insert( 0,os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the breadboard module
from breadboard import BreadboardClient

## Get parameters for images

In [6]:
# initialize the client
client = BreadboardClient(config_path='API_CONFIG.json')

In [7]:
imagenames = ['10-09-2018_00_21_57_TopA','10-09-2018_00_21_57_TopB']
params = ['FB_field_13_V','ImagFreq0', 'cylinder hold time']
df = client.get_images_df(imagenames, params)

In [12]:
df.head()

,imagename,ImagFreq0,cylinder hold time,url,tags,notes,filepath,id,created,total_atoms,FB_field_13_V,atom,cropi
0,10-09-2018_00_21_57_TopA,193.95,0.2,http://breadboard-215702.appspot.com/images/10/,NaN,tes232t,NaN,10.0,2018-10-09T00:21:57Z,NaN,3.774,Li,NaN
1,10-09-2018_00_21_57_TopB,193.95,0.2,http://breadboard-215702.appspot.com/images/11/,NaN,None,NaN,11.0,2018-10-09T00:21:57Z,NaN,3.774,Li,NaN


In [ ]:
# note the API does a lot of caching so the data might be a little outdated

## Update notes for a single image

In [9]:
params_update  = {
            'notes': "test"
        }

In [10]:
resp = client.update_image( df.at[0,'id'],df.at[0,'imagename'], params_update)

In [11]:
resp.status_code

200